[Reference](https://medium.com/@deasadiqbal/transforming-machine-learning-models-into-apis-in-python-f9af614ab499)

# Flask — A Web Services’ Framework in Python

In [1]:
from flask import Flask

app = Flask(__name__)


@app.route("'/landing-page', methods=['POST']")
def test():
    return "Welcome to machine learning model APIs!"


if __name__ == '__main__':
    app.run(debug=True)

# Scikit-learn Models with Flask

In [6]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

# Load the dataset from a URL and select only the specified features
url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
df = pd.read_csv(url)
selected_features = ['Age', 'Sex', 'Embarked', 'Survived']  # We're interested in these four features
df = df[selected_features]

# Data Preprocessing
categorical_columns = []  # To store categorical columns
for column, data_type in df.dtypes.iteritems():
    if data_type == 'O':  # Check if the column has object data type (categorical)
        categorical_columns.append(column)
    else:
        df[column].fillna(0, inplace=True)  # Fill missing values in non-categorical columns with 0

# Perform one-hot encoding on categorical columns
df_encoded = pd.get_dummies(df, columns=categorical_columns, dummy_na=True)

# Define the dependent variable and split data into features (x) and target (y)
dependent_variable = 'Survived'
x = df_encoded[df_encoded.columns.difference([dependent_variable])]
y = df_encoded[dependent_variable]

# Create and train a Logistic Regression classifier
lr = LogisticRegression()
lr.fit(x, y)

import joblib
joblib.dump(lr, 'lr_model.pkl')

<ipython-input-6-5af60d2fe7df>:14: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, data_type in df.dtypes.iteritems():


['lr_model.pkl']

In [7]:
lr = joblib.load('lr_model.pkl')
lr

LogisticRegression()

In [9]:
from flask import Flask, jsonify
import requests

app = Flask(__name__)
@app.route('/predict', methods=['POST'])
def predict():
     json_ = requests.json
     query_df = pd.DataFrame(json_)
     query = pd.get_dummies(query_df)
     prediction = lr.predict(query)
     return jsonify({'prediction': list(prediction)})

In [10]:
from flask import Flask, jsonify
import requests
import traceback
import sys

app = Flask(__name__)

@app.route('/predict', methods=['POST']) # Your API endpoint URL would consist /predict
def predict():
    if lr:
        try:
            json_ = requests.json
            query = pd.get_dummies(pd.DataFrame(json_))
            query = query.reindex(columns=model_columns, fill_value=0)

            prediction = list(lr.predict(query))

            return jsonify({'prediction': prediction})

        except:

            return jsonify({'trace': traceback.format_exc()})
    else:
        print ('Train the model first')
        return ('No model here to use')

if __name__ == '__main__':
    try:
        port = int(sys.argv[1]) # This is for a command-line argument
    except:
        port = 12345 # If you don't provide any port then the port will be set to 12345
    lr = joblib.load(model_file_name) # Load "lr_model.pkl"
    print ('Model loaded')

    model_columns = joblib.load(model_columns_file_name) # Load "r_lmodel_columns.pkl"
    print ('Model columns loaded')
    app.run(port=port, debug=True)